##### Setup
Error: the datatype for boolean values in google sheet seems to be either unretrievable or (if changed to 1 or 0) converted to float which would give an error in model run, if this can be fix

In [8]:
import sys
sys.path.append('../src')



# data Analysis 
import geopandas as gpd
import pandas as pd
import numpy as np



# plot
import matplotlib.pyplot as plt
import gc

# output
from output_image import write_output


# multiprocessing
import multiprocessing
import brun


# model
from school_model import School

# spreadsheets
# import spreadsheets

#config
import configparser
import warnings
warnings.filterwarnings("ignore")

# Parameter Setup
Here using a test trial small schedule with 5 steps of class per day
<br>
Run for 15 days

In [13]:
school_params = '../config/schoolparams.ini'
parser = configparser.ConfigParser()
parser.read(school_params)

map_path = parser['SHAPEFILE']['path']
schedule_path = parser['SCHEDULE']['path'] #small schedule has only 5 steps per day for testing purposes
#full day schedule should be "day_schedule.csv" 
schedule_steps = int(parser['TIME']['schedule_steps']) # full day_schedule steps should be 90


# if you wish to use google sheet parameter input, you'll have to follow instructions for getting credentials of the sheet api
# follow the first two steps in:
# https://developers.google.com/sheets/api/quickstart/python
# and save your credentials in the src folder as listed below

# SHEET_URL = 'https://docs.google.com/spreadsheets/d/1Quyyey5B_kdQK1_OU0OkIDGZE27tuUfjZ6hCsV99-sM'
# credentials = '../src/credentials.json'


####Pulling in parameters from config files####
population = parser['SCHOOL_POPULATION']
grade_N = int(population['grade_N'])
KG_N = int(population['KG_N'])
preschool_N = int(population['preschool_N'])
special_education_N = int(population['special_education_N'])
faculty_N = int(population['faculty_N'])

mask_prob = float(population['mask_prob'])
init_patient = int(population['init_patient'])
attend_rate = int(population['population_attending'])

days = int(parser['TIME']['days'])
max_steps = days*schedule_steps
iterations = 1

#This in the process of becoming a user adjustable parameter, 
#Hence, the alterative file path. 

activity_params = '../config/activities.ini'
parser_activity = configparser.ConfigParser()
parser_activity.read(activity_params)
seat_dist = int(parser_activity['CLASSROOM_REGULAR']['seat_dist'])


In [14]:
# fixed parameters are model input parameters that are static for all batch running jobs
fixed_parameters = {}


fixed_parameters['grade_N'] = grade_N
fixed_parameters['KG_N'] = KG_N
fixed_parameters['preschool_N'] = preschool_N
fixed_parameters['special_education_N'] = special_education_N
fixed_parameters['faculty_N'] = faculty_N
fixed_parameters['seat_dist'] = seat_dist
fixed_parameters['init_patient'] = population['init_patient']

# same logic as fixed parameters, only these will be batch ran for all combinations 
variable_parameters = {}
variable_parameters['attend_rate'] = [1]
variable_parameters['inclass_lunch'] = [True]
variable_parameters['mask_prob'] = [0, 0.25]

In [15]:
# dataframe parameter setup that can port with google sheet
# TODO: currently the sheet script cannot retrieve boolean values so this input method is not fully functioning
# namely inclass_lunch
# should be an easy fix

# param_df = spreadsheets.return_params_df(sheet_url=SHEET_URL, credentials=credentials)

# Single Run

In [ ]:
school = School(map_path, schedule_path, grade_N, KG_N, preschool_N, special_education_N, 
                 faculty_N, seat_dist, init_patient=init_patient, attend_rate=attend_rate, mask_prob=mask_prob, inclass_lunch=True, username="jleiucsd")


while school.running and school.schedule.steps < max_steps:
    school.step()

params = "{'test': 0}" 
agent_df = school.datacollector.get_agent_vars_dataframe()
model_df = school.datacollector.get_model_vars_dataframe()

1


# Multiprocessing Run

In [17]:
if __name__ ==  '__main__': 
    # for more on multiprocessing, check src/brun 
    batchrunner = brun.BatchrunnerMP(model=School, 
                                     map_path=map_path,
                                     schedule_path=schedule_path,
                                     max_steps=max_steps, 
                                     iterations=iterations, 
                                     fixed_parameters=fixed_parameters, 
                                     variable_parameters=variable_parameters)
    results = batchrunner.run_all()

Your system has 4 available processors.


TypeError: '<=' not supported between instances of 'int' and 'str'

In [ ]:
for params, dfs in results.items():
    write_output(params, dfs, map_path)